<a href="https://colab.research.google.com/github/bryanbayup/phising-detection/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install seqeval tensorflow==2.8.0 tensorflow-addons protobuf==3.20.0

In [3]:
# # Download FastText
!wget -O id.tar.gz "https://www.dropbox.com/scl/fi/sju4o3keikox69euw51vy/id.tar.gz?rlkey=5jr3ijtbdwfahq7xcgig28qvy&e=1&st=gntzkzeo&dl=1"
!tar -xzf id.tar.gz

--2024-12-06 07:44:48--  https://www.dropbox.com/scl/fi/sju4o3keikox69euw51vy/id.tar.gz?rlkey=5jr3ijtbdwfahq7xcgig28qvy&e=1&st=gntzkzeo&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uca3416caccb22fde9b99306f935.dl.dropboxusercontent.com/cd/0/inline/CfvzgSa_9BXK5ZFo7sfhEoW4dlcptu1FEhWiGz2M_tz_3U7PrkroThZ6heGEB9OpzY666Ysr2x-MFUuQLzSnNHaQbS7q8TcqccWwgMN1Y24V_EKOvCl-9azVCsPFlrEO0y0/file?dl=1# [following]
--2024-12-06 07:44:49--  https://uca3416caccb22fde9b99306f935.dl.dropboxusercontent.com/cd/0/inline/CfvzgSa_9BXK5ZFo7sfhEoW4dlcptu1FEhWiGz2M_tz_3U7PrkroThZ6heGEB9OpzY666Ysr2x-MFUuQLzSnNHaQbS7q8TcqccWwgMN1Y24V_EKOvCl-9azVCsPFlrEO0y0/file?dl=1
Resolving uca3416caccb22fde9b99306f935.dl.dropboxusercontent.com (uca3416caccb22fde9b99306f935.dl.dropboxusercontent.com)... 162.125.3.15, 2620:100:6018:15::a2

In [2]:
import json
import os
import re
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from seqeval.metrics import classification_report as seq_classification_report

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [12]:
!pip install Sastrawi gensim

In [13]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import gensim
from gensim.models import KeyedVectors

In [14]:
# Text cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Memuat stopwords
with open('stopword_list_tala.txt', 'r', encoding='utf-8') as f:
    stop_words = f.read().splitlines()
stop_words = set(word.strip().lower() for word in stop_words)  # Pastikan lowercase dan tanpa spasi

# Stemming dengan Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi preprocessing teks
def preprocess_text(text):
    # Konversi teks menjadi huruf kecil dan hapus tanda baca
    text = clean_text(text)
    # Tokenisasi
    tokens = text.split()
    # Penghapusan stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    tokens_after_stemming = [stemmer.stem(token) for token in tokens]
    # Gabungkan kembali tokens menjadi string
    text = ' '.join(tokens_after_stemming)
    return text

In [15]:
fasttext_model = KeyedVectors.load_word2vec_format('id.vec', binary=False)

In [16]:
data_path = 'data2.json'  # ganti dengan nama file dataset Anda
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Ekstrak user turns
utterances = []
intents = []
entities_all = []

for conv in data:
    for turn in conv['turns']:
        if turn['speaker'] == 'user':
            utt = turn['utterance']
            intent = turn['intent']
            ents = turn.get('entities', [])
            utterances.append(utt)
            intents.append(intent)
            entities_all.append(ents)

df = pd.DataFrame({'utterance': utterances, 'intent': intents, 'entities': entities_all})
df['utterance_clean'] = df['utterance'].apply(preprocess_text)

# Label intent
intent_labels = sorted(df['intent'].unique().tolist())
intent_label2id = {l:i for i,l in enumerate(intent_labels)}
intent_id2label = {i:l for l,i in intent_label2id.items()}
df['intent_id'] = df['intent'].map(intent_label2id)

texts = df['utterance_clean'].tolist()
Y_intent = df['intent_id'].values

tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='')
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
vocab_size = len(word_index)+1

seqs = tokenizer.texts_to_sequences(texts)
max_seq_length = max(len(s) for s in seqs)
max_seq_length = min(max_seq_length, 60)  # tingkatkan max length jika perlu
X_padded = pad_sequences(seqs, maxlen=max_seq_length, padding='post')

# NER Labeling
all_ner_labels = set(['O'])
for ents in entities_all:
    for e in ents:
        ent_type = e['entity']
        all_ner_labels.add("B-"+ent_type)
        all_ner_labels.add("I-"+ent_type)
all_ner_labels = sorted(list(all_ner_labels))
ner_label2id = {l:i for i,l in enumerate(all_ner_labels)}
ner_id2label = {i:l for l,i in ner_label2id.items()}

def create_ner_labels(texts, entities_all, tokenizer, max_len):
    all_ner_seq = []
    for txt, ents in zip(texts, entities_all):
        tok_seq = tokenizer.texts_to_sequences([txt])[0]
        ner_seq = ['O']*len(tok_seq)
        words = txt.split()
        for e in ents:
            ent_value = preprocess_text(e['value'])
            ent_words = ent_value.split()
            for i in range(len(words)-len(ent_words)+1):
                if words[i:i+len(ent_words)] == ent_words:
                    for w_j in range(len(ent_words)):
                        if i+w_j < len(ner_seq):
                            ner_seq[i+w_j] = 'B-'+e['entity'] if w_j == 0 else 'I-'+e['entity']
                    break
        if len(ner_seq) > max_len:
            ner_seq = ner_seq[:max_len]
        else:
            ner_seq += ['O']*(max_len-len(ner_seq))
        all_ner_seq.append(ner_seq)
    return all_ner_seq

ner_labels = create_ner_labels(df['utterance_clean'].tolist(), df['entities'].tolist(), tokenizer, max_seq_length)
ner_ids = [[ner_label2id[l] for l in seq] for seq in ner_labels]
Y_ner = np.array(ner_ids) # Nanti CRF pakai sparse target

X_train, X_val, Y_train_intent, Y_val_intent, Y_train_ner, Y_val_ner = train_test_split(
    X_padded, Y_intent, Y_ner, test_size=0.2, random_state=42, stratify=Y_intent
)

# Embedding
embedding_dim = fasttext_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in word_index.items():
    if word in fasttext_model:
        embedding_matrix[idx] = fasttext_model[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

In [18]:
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout, Layer
from tensorflow.keras.models import Model
import tensorflow_addons as tfa

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.dense = Dense(1)  # Define Dense layer here

    def call(self, inputs):
        # inputs: (batch, seq, dim)
        # Compute attention score
        w = self.dense(inputs)  # (batch, seq, 1)
        a = tf.nn.softmax(w, axis=1)  # attention weights
        context = tf.reduce_sum(a * inputs, axis=1)  # (batch, dim)
        return context

num_intent = len(intent_label2id)
num_ner = len(ner_label2id)

input_layer = Input(shape=(max_seq_length,))
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=max_seq_length,
    trainable=True
)(input_layer)

x = Bidirectional(LSTM(128, return_sequences=True))(embedding_layer)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Dropout(0.5)(x)

# Attention untuk intent
context = AttentionLayer()(x)  # Use the custom AttentionLayer
intent_out = Dense(num_intent, activation='softmax', name='intent_output')(context)

# NER dengan CRF
ner_dense = Dense(num_ner)(x)
# Kita butuh Custom CRF layer, tapi di tensorflow-addons sudah disediakan fungsi CRF ops.
# Akan gunakan CRF layer dari tfa.layers.CRF
from tensorflow_addons.text.crf import crf_decode

logits = Dense(num_ner)(x)
# CRF param
trans_params = tf.Variable(tf.random.uniform(shape=(num_ner, num_ner)))
# Akan gunakan model compile custom training loop

# Kita buat model tanpa compile dulu, karena CRF butuh custom train_step
model = Model(inputs=input_layer, outputs=[intent_out, logits])

In [20]:
# Kita akan override train_step agar CRF loss bisa dihitung
optimizer = tf.keras.optimizers.Adam()
loss_fn_intent = tf.keras.losses.SparseCategoricalCrossentropy()

# Assign the optimizer to the model
model.compile(optimizer=optimizer, loss={'intent_output': loss_fn_intent}) # loss for intent is specified here

@tf.function
def train_step(X, Y_intent, Y_ner):
    with tf.GradientTape() as tape:
        intent_pred, ner_logits = model(X, training=True)
        # Intent loss
        loss_intent = loss_fn_intent(Y_intent, intent_pred)

        # CRF loss
        log_likelihood, _ = tfa.text.crf.crf_log_likelihood(
            ner_logits, Y_ner, tf.reduce_sum(tf.cast(X!=0, tf.int32), axis=1), trans_params
        )
        loss_ner = -tf.reduce_mean(log_likelihood)

        loss = loss_intent + loss_ner

    grads = tape.gradient(loss, model.trainable_variables+[trans_params])
    model.optimizer.apply_gradients(zip(grads, model.trainable_variables+[trans_params])) # model.optimizer now refers to the Adam optimizer
    return loss, loss_intent, loss_ner

@tf.function
def val_step(X, Y_intent, Y_ner):
    intent_pred, ner_logits = model(X, training=False)
    loss_intent = loss_fn_intent(Y_intent, intent_pred)
    log_likelihood, _ = tfa.text.crf.crf_log_likelihood(
        ner_logits, Y_ner, tf.reduce_sum(tf.cast(X!=0, tf.int32), axis=1), trans_params
    )
    loss_ner = -tf.reduce_mean(log_likelihood)
    loss = loss_intent + loss_ner
    return loss, loss_intent, loss_ner, intent_pred, ner_logits

# Simple training loop
batch_size = 16
epochs = 10
num_train = X_train.shape[0]
num_val = X_val.shape[0]
train_steps = num_train//batch_size
val_steps = num_val//batch_size

best_val_loss = np.inf
patience = 3
no_improve = 0

for epoch in range(1, epochs+1):
    # Shuffle train
    idx = np.arange(num_train)
    np.random.shuffle(idx)
    X_train_s = X_train[idx]
    Y_train_intent_s = Y_train_intent[idx]
    Y_train_ner_s = Y_train_ner[idx]

    # Training
    train_loss = 0
    for i in range(train_steps):
        batch_X = X_train_s[i*batch_size:(i+1)*batch_size]
        batch_intent = Y_train_intent_s[i*batch_size:(i+1)*batch_size]
        batch_ner = Y_train_ner_s[i*batch_size:(i+1)*batch_size]
        loss, li, ln = train_step(batch_X, batch_intent, batch_ner)
        train_loss += loss.numpy()
    train_loss /= train_steps

    # Validation
    val_loss_cum = 0
    all_intent_pred = []
    all_intent_true = []
    all_ner_pred = []
    all_ner_true = []
    for i in range(val_steps):
        batch_X = X_val[i*batch_size:(i+1)*batch_size]
        batch_intent = Y_val_intent[i*batch_size:(i+1)*batch_size]
        batch_ner = Y_val_ner[i*batch_size:(i+1)*batch_size]
        vloss, vli, vln, intent_p, ner_logits = val_step(batch_X, batch_intent, batch_ner)
        val_loss_cum += vloss.numpy()
        # Intent eval
        ip = np.argmax(intent_p.numpy(), axis=1)
        all_intent_pred.extend(ip.tolist())
        all_intent_true.extend(batch_intent.tolist())
        # NER decode
        seq_len = np.sum(batch_X!=0,axis=1)
        viterbi_paths = []
        for j in range(batch_X.shape[0]):
            logit_seq = ner_logits[j,:seq_len[j],:]
            viterbi_path, _ = tfa.text.crf.viterbi_decode(logit_seq, trans_params)
            viterbi_paths.append(viterbi_path)
        all_ner_pred.extend(viterbi_paths)
        all_ner_true.extend([ner[:l] for ner,l in zip(batch_ner, seq_len)])
    val_loss = val_loss_cum/val_steps

    intent_acc = np.mean(np.array(all_intent_pred) == np.array(all_intent_true))*100

    # Convert ner to labels
    pred_tag = [[ner_id2label[l] for l in seq] for seq in all_ner_pred]
    true_tag = [[ner_id2label[l] for l in seq] for seq in all_ner_true]

    print(f"Epoch {epoch}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Intent Acc: {intent_acc:.2f}%")
    print(seq_classification_report(true_tag, pred_tag))

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improve = 0
        # Simpan model weights
        model.save_weights('best_multitask_model.h5')
        np.save('best_trans_params.npy', trans_params.numpy())
    else:
        no_improve += 1
        if no_improve >= patience:
            print("Early Stopping...")
            break

# Load best weights
model.load_weights('best_multitask_model.h5')
trans_params.assign(np.load('best_trans_params.npy'))

Epoch 1/10
Train Loss: 10.4508, Val Loss: 9.1661, Intent Acc: 56.25%
              precision    recall  f1-score   support

      animal       0.00      0.00      0.00        10
   condition       0.00      0.00      0.00         2
    location       0.00      0.00      0.00         3
      status       0.00      0.00      0.00         1
     symptom       0.17      0.33      0.23        18

   micro avg       0.17      0.18      0.17        34
   macro avg       0.03      0.07      0.05        34
weighted avg       0.09      0.18      0.12        34



/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/10
Train Loss: 8.2152, Val Loss: 7.3527, Intent Acc: 56.25%
              precision    recall  f1-score   support

      animal       0.48      1.00      0.65        10
   condition       0.00      0.00      0.00         2
    location       0.00      0.00      0.00         3
      status       0.00      0.00      0.00         1
     symptom       0.26      0.39      0.31        18

   micro avg       0.35      0.50      0.41        34
   macro avg       0.15      0.28      0.19        34
weighted avg       0.28      0.50      0.35        34

Epoch 3/10
Train Loss: 6.9263, Val Loss: 6.6060, Intent Acc: 50.00%
              precision    recall  f1-score   support

      animal       1.00      0.60      0.75        10
   condition       0.00      0.00      0.00         2
    location       0.33      0.33      0.33         3
      status       0.00      0.00      0.00         1
     symptom       0.26      0.28      0.27        18

   micro avg       0.43      0.35      0.39      

<tf.Variable 'UnreadVariable' shape=(11, 11) dtype=float32, numpy=
array([[ 0.6593117 ,  0.43100312, -0.00389422,  0.07282911,  0.81275475,
         0.24689274,  0.12578773,  0.6911004 ,  0.85904115,  0.5607759 ,
         0.24619196],
       [ 0.74130625,  0.25703546,  0.23628859,  0.6759545 ,  0.68496174,
         0.6025608 ,  0.3355796 , -0.00960143,  0.90292907, -0.0031366 ,
         0.16572301],
       [ 0.09484031,  0.23557076,  0.39058518,  0.25497198,  0.24212798,
         0.58641374,  0.189031  ,  0.93358505,  0.04939082,  0.14150749,
        -0.01871487],
       [-0.01139066,  0.9482258 ,  0.19912736,  0.38568377,  0.7095453 ,
         0.33143774,  0.3341312 ,  0.27803013,  0.750947  ,  0.73537177,
         0.3380729 ],
       [ 0.2609268 ,  0.25051278,  0.41061822,  0.4529809 ,  0.83135384,
         0.8879625 ,  0.32480913,  0.31689855,  0.3299686 ,  0.6131312 ,
         0.91074073],
       [ 0.8167935 ,  0.28066316,  0.9565467 ,  0.5284629 ,  0.4354476 ,
         0.0733595 ,

In [21]:
class DialogManager:
    def __init__(self):
        self.state = 'IDLE'
        self.reported_animal = None
        self.symptoms = []
        self.consultation_recommended = False

    def predict_intent_ner(self, user_input):
        user_clean = preprocess_text(user_input)
        seq = tokenizer.texts_to_sequences([user_clean])
        seq_padded = pad_sequences(seq, maxlen=max_seq_length, padding='post')
        intent_p, ner_logits = model(seq_padded, training=False)
        intent_id = np.argmax(intent_p.numpy())
        intent = intent_id2label[intent_id]

        length = np.sum(seq_padded!=0, axis=1)[0]
        logit_seq = ner_logits[0,:length,:]
        viterbi_path, _ = tfa.text.crf.viterbi_decode(logit_seq, trans_params)
        entities = []
        tokens = user_clean.split()
        for i, l_id in enumerate(viterbi_path):
            lbl = ner_id2label[l_id]
            if lbl != 'O':
                ent_type = lbl.split('-')[1]
                entities.append({'entity': ent_type, 'value': tokens[i]})
        return intent, entities

    def get_response(self, user_input):
        intent, entities = self.predict_intent_ner(user_input)

        # Contoh logika multi-turn sederhana
        if intent == "Greeting":
            return "Halo! Ada yang bisa saya bantu hari ini?"
        elif intent == "Thanks":
            return "Sama-sama! Senang membantu."
        elif intent == "Rekomendasi Penanganan Awal":
            # Periksa entitas
            animal_ents = [e for e in entities if e['entity']=='animal']
            symptom_ents = [e for e in entities if e['entity']=='symptom']
            if animal_ents:
                self.reported_animal = animal_ents[0]['value']
            if symptom_ents:
                self.symptoms.extend(s['value'] for s in symptom_ents)

            # Jika banyak gejala, mungkin rekomendasikan dokter
            if len(self.symptoms) > 2 and not self.consultation_recommended:
                self.consultation_recommended = True
                return f"Saya mencatat {self.reported_animal if self.reported_animal else 'hewan'} dengan gejala {', '.join(self.symptoms)}. " \
                       "Kondisi tampak serius. Sebaiknya konsultasikan dengan dokter hewan."

            return f"Saya mencatat {self.reported_animal if self.reported_animal else 'hewan'} dengan gejala {', '.join(self.symptoms)}. " \
                   "Pastikan kondisi lingkungan nyaman dan jika tidak membaik, hubungi dokter hewan."
        else:
            return "Maaf, saya belum mengerti maksud Anda."

dm = DialogManager()
print(dm.get_response("Halo, saya punya pertanyaan tentang anjing saya."))
print(dm.get_response("Anjing saya muntah dan diare sejak kemarin."))

Saya mencatat hewan dengan gejala . Pastikan kondisi lingkungan nyaman dan jika tidak membaik, hubungi dokter hewan.
Maaf, saya belum mengerti maksud Anda.


In [24]:
print(dm.get_response("Kucing saya terlihat lesu dan kehilangan nafsu makan. Apa yang harus saya lakukan?"))

Saya mencatat kucing dengan gejala lesu, hilang, nafsu, makan. Kondisi tampak serius. Sebaiknya konsultasikan dengan dokter hewan.


In [25]:
print(dm.get_response("Anjing saya terlihat pincang setelah bermain. Apa langkah awal yang harus dilakukan?"))

Saya mencatat anjing dengan gejala lesu, hilang, nafsu, makan, pincang. Pastikan kondisi lingkungan nyaman dan jika tidak membaik, hubungi dokter hewan.
